In [1]:
import sys, os, _pickle as pickle
import tensorflow as tf
import numpy as np
from tensorflow.contrib import learn

In [2]:
data_dir = 'data'
ckpt_dir = 'checkpoint'
word_embd_dir = 'checkpoint/word_embd_100'
model_dir = 'checkpoint/model'

In [3]:
f = open('train_data.pkl', 'rb')
data, relation, e1, e2, e1_pos, e2_pos = pickle.load(f)

In [4]:
max_document_length = max([len(x.split(" ")) for x in data])

In [5]:
word_embd_dim = 100
pos_embd_dim = 25
dep_embd_dim = 25
word_vocab_size = 400000
pos_vocab_size = 10
dep_vocab_size = 10
relation_classes = 19
state_size = 100
max_sequence_length = 97
batch_size = 10
channels = 3
lambda_l2 = 0.0001

In [7]:
with tf.name_scope("input"):
    sequence_length = tf.placeholder(tf.int32, shape=[batch_size], name="sequence_length")
    word_ids = tf.placeholder(tf.int32, shape=[batch_size, None], name="word_ids")
    pos_ids = tf.placeholder(tf.int32, [batch_size, None], name="pos_ids")
    dep_ids = tf.placeholder(tf.int32, [batch_size, None], name="dep_ids")
    y = tf.placeholder(tf.int32, [batch_size, None], name="y")

In [9]:
with tf.name_scope("word_embedding"):
    W = tf.Variable(tf.constant(0.0, shape=[word_vocab_size, word_embd_dim]), name="W")
    embedding_placeholder = tf.placeholder(tf.float32,[word_vocab_size, word_embd_dim])
    embedding_init = W.assign(embedding_placeholder)
    embedded_word = tf.nn.embedding_lookup(W, word_ids)
    word_embedding_saver = tf.train.Saver({"word_embedding/W": W})

In [10]:
with tf.name_scope("pos_embedding"):
    W = tf.Variable(tf.random_uniform([pos_vocab_size, pos_embd_dim]), name="W")
    embedded_pos = tf.nn.embedding_lookup(W, pos_ids)
    pos_embedding_saver = tf.train.Saver({"pos_embedding/W": W})

In [11]:
with tf.name_scope("dep_embedding"):
    W = tf.Variable(tf.random_uniform([dep_vocab_size, dep_embd_dim]), name="W")
    embedded_dep = tf.nn.embedding_lookup(W, dep_ids)
    dep_embedding_saver = tf.train.Saver({"dep_embedding/W": W})

In [12]:
hidden_states = tf.zeros([channels, batch_size, state_size], name='hidden_state')
cell_states = tf.zeros([channels, batch_size, state_size], name='cell_state')

In [13]:
init_states = [tf.contrib.rnn.LSTMStateTuple(hidden_states[i], cell_states[i]) for i in range(channels)]

In [15]:
with tf.variable_scope("word_lstm"):
    cell = tf.contrib.rnn.BasicLSTMCell(state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_word, sequence_length=sequence_length, initial_state=init_states[0])
    state_series_word = tf.reduce_max(state_series, axis=1)

In [16]:
with tf.variable_scope("pos_lstm"):
    cell = tf.contrib.rnn.BasicLSTMCell(state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_pos, sequence_length=sequence_length,initial_state=init_states[1])
    state_series_pos = tf.reduce_max(state_series, axis=1)

In [17]:
with tf.variable_scope("dep_lstm"):
    cell = tf.contrib.rnn.BasicLSTMCell(state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_dep, sequence_length=sequence_length, initial_state=init_states[2])
    state_series_dep = tf.reduce_max(state_series, axis=1)

In [19]:
state_series = tf.concat([state_series_word, state_series_pos, state_series_dep], 1)

In [20]:
with tf.name_scope("hidden_layer"):
    W = tf.Variable(tf.truncated_normal([channels * state_size, relation_classes], -0.1, 0.1), name="W")
    b = tf.Variable(tf.zeros([relation_classes]), name="b")
    logits = tf.matmul(state_series, W) + b
    predictions = tf.argmax(logits, 1)

In [21]:
global_step = tf.Variable(0, name="global_step")

In [22]:
tv_all = tf.trainable_variables()
tv_regu = []
non_reg = ["word_embedding/W:0","pos_embedding/Variable:0","global_step:0"]
for t in tv_all:
    if t.name not in non_reg:
        tv_regu.append(t)

In [24]:
with tf.name_scope("loss"):
    l2_loss = lambda_l2 * tf.reduce_sum([ tf.nn.l2_loss(v) for v in tv_regu ])
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
    total_loss = loss + l2_loss

In [25]:
optimizer = tf.train.AdamOptimizer(0.001).minimize(total_loss, global_step=global_step)

In [26]:
sess = tf.Session()

In [27]:
sess.run(tf.global_variables_initializer())

In [28]:
latest_embd = tf.train.latest_checkpoint(word_embd_dir)
word_embedding_saver.restore(sess, latest_embd)

INFO:tensorflow:Restoring parameters from checkpoint/word_embd_100/word_embd


In [31]:
num_batches = int(len(data)/batch_size)

In [32]:
num_batches

800

In [35]:
f = open('data/vocab.pkl', 'rb')
vocab = pickle.load(f)

In [36]:
char2id = dict((c, i) for i,c in enumerate(vocab))
id2char = dict((i, c) for i,c in enumerate(vocab))

In [38]:
def char2id_func(word):
    try:
        id = char2id[word]
    except Exception:
        id = word_vocab_size + 1
    return id